In [1]:
import cosima_cookbook as cc
from cosima_cookbook import distributed as ccd
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import glob,os
import cmocean.cm as cmocean

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

In [2]:
client = Client(memory_limit='500GiB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 21,Total memory: 3.42 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42851,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 21
Started: Just now,Total memory: 3.42 TiB
Comm: tcp://127.0.0.1:35345,Total threads: 3
Dashboard: /proxy/45431/status,Memory: 500.00 GiB
Nanny: tcp://127.0.0.1:43907,


In [3]:
	session = cc.database.create_session()
	
	
	imm=8
	monthdays = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
	month = str(int(imm))
	month = month.zfill(2)
	year = str(1964)
	start_time='1958-01'
	end_time='2018-12'
	#Start_time0 and end_time0 are for importing the daily transport, and it ahsthe number of days in the month    
	start_time0=start_time+'-01'     
	end_time0=end_time +'-31'
	print(start_time0) 
	print(end_time0) 
	exp = '01deg_jra55v140_iaf_cycle3'
	
	print("Start date =" + start_time) 
	year2=str(int(start_time[0:4])+1)
	month2=str(int(start_time[5:7])+1)
	month2=str(int(month2))
	month2 = month2.zfill(2)    
	print("month2 is =" + month2) 
	print("year2 is =" + str(year2))     
	
	imon = int(imm)
	if imon <12:
		end_time=year+'-'+month2
	else:
		end_time=year2+'-01'
	
	print("End date =" + end_time) 
	
	
	time_period = str(int(start_time[:4]))+'-'+str(int(end_time[:4]))
	
	# reference density value:
	rho_0 = 1035.0
	# specific heat capacity of sea water:
	cp = 3992.1
	lat_range = slice(-90,-59)
	
	isobath_depth = 1000
	
	# pick a freezing point temp:
	temp_freezing = -3.82
	
	

1958-01-01
2018-12-31
Start date =1958-01
month2 is =02
year2 is =1959
End date =1964-02


Comment cell below if already saved the data

In [4]:
# #Improting from simulation
# v= cc.querying.getvar(exp,'v',session,frequency='1 monthly',
#         chunks={'yu_ocean': '400GB', 'xu_ocean': '400GB'}).sel(yu_ocean=lat_range).groupby('time.month').mean()

# u= cc.querying.getvar(exp,'u',session,frequency='1 monthly',
#         chunks={'yu_ocean': '400GB', 'xu_ocean': '400GB'}).sel(yu_ocean=lat_range).groupby('time.month').mean()

# temp = cc.querying.getvar(exp,'temp',session,frequency='1 monthly',
#         chunks={'yt_ocean': '200MB', 'xt_ocean': '200MB'}).sel(yt_ocean=lat_range).groupby('time.month').mean()  -273.15

# #loading and calculating climatology
# #u
# u.load()
# u.to_netcdf('/home/156/wf4500/x77_wf4500/ASC_project/Post_process/IAF_u_climatology.nc')
# del u
# #v
# v.load()
# v.to_netcdf('/home/156/wf4500/x77_wf4500/ASC_project/Post_process/IAF_v_climatology.nc')
# del v
# #dzt
# dzt= cc.querying.getvar(exp,'dzt',session,frequency='1 monthly',
#         chunks={'yt_ocean': '400GB', 'xt_ocean': '400GB'}).sel(yt_ocean=lat_range).groupby('time.month').mean()
# dzt.load()
# dzt.to_netcdf('/home/156/wf4500/x77_wf4500/ASC_project/Post_process/IAF_dzt_climatology.nc')
# del dzt
# #temp
# temp.load()
# temp.to_netcdf('/home/156/wf4500/x77_wf4500/ASC_project/Post_process/IAF_temp_climatology.nc')
# del temp

In [5]:
#reloading all data
temp=xr.open_dataset('/home/156/wf4500/x77_wf4500/ASC_project/Post_process/IAF_temp_climatology.nc')
u=xr.open_dataset('/home/156/wf4500/x77_wf4500/ASC_project/Post_process/IAF_u_climatology.nc')
v=xr.open_dataset('/home/156/wf4500/x77_wf4500/ASC_project/Post_process/IAF_v_climatology.nc')
dzt=xr.open_dataset('/home/156/wf4500/x77_wf4500/ASC_project/Post_process/IAF_dzt_climatology.nc')

In [6]:
#Regridding u and bv towards the t grid.

#reggridding u
ut=u.interp(yu_ocean=temp.yt_ocean)
#reggridding v
vt=v.interp(xu_ocean=temp.xt_ocean)


#temp_x and tempy
tempx=temp.interp(xt_ocean=u.xu_ocean)
tempy=temp.interp(yt_ocean=v.yu_ocean)



#reggridding dzt to xgrid
dztx=dzt.interp(xt_ocean=u.xu_ocean)
#reggridding dzt to ygrid
dzty=dzt.interp(yt_ocean=v.yu_ocean)



In [7]:
# the transports will be

# #uhrho
uhrho=ut.u*dztx.dzt*rho_0

# #vhrho
vhrho=vt.v*dzty.dzt*rho_0


In [11]:
	
	import os
	#outpath = '/g/data/x77/wf4500/ASC_project/model_data/access-om2/'+exp+'/Antarctic_cross_slope/Daily/uhrho_vhrho_'+start_time+'.nc'
	#ds = xr.open_dataset(outpath)
#	vhrho_nt = ds['vhrho_nt']#
#	uhrho_et = ds['uhrho_et']
	
	# subtract freezing point heat transport:
	yt_ocean = cc.querying.getvar('01deg_jra55v13_ryf9091','yt_ocean',session,n=1)
	xt_ocean = cc.querying.getvar('01deg_jra55v13_ryf9091','xt_ocean',session,n=1)
	xu_ocean = cc.querying.getvar('01deg_jra55v13_ryf9091','xu_ocean',session,n=1)
	dxu = cc.querying.getvar('01deg_jra55v13_ryf9091','dxu',session,n=1)
	dyt = cc.querying.getvar('01deg_jra55v13_ryf9091','dyt',session,n=1)
	# give dxu and dyt correct coordinates:
	dxu.coords['nj'] = yt_ocean.values
	dxu.coords['ni'] = xt_ocean['xt_ocean'].values
	dxu = dxu.rename(({'ni':'xt_ocean', 'nj':'yt_ocean'}))
	dyt.coords['nj'] = yt_ocean.values
	dyt.coords['ni'] = xt_ocean['xt_ocean'].values
	dyt = dyt.rename(({'ni':'xt_ocean', 'nj':'yt_ocean'}))
	# select latitude range:
	dxu = dxu.sel(yt_ocean=lat_range).interp(yt_ocean=v.yu_ocean).load()
	dyt = dyt.sel(yt_ocean=lat_range).interp(xt_ocean=u.xu_ocean).load()
	
	# Note that in newer mom5 versions this could also be done with ty_trans_int_z, 
	# but there is a problem with this diagnostic in older runs, and even
	# using ty_trans, there is a slight difference. Not sure why?
	
	# Note vhrho_nt is v*dz*1035 and is positioned on north centre edge of t-cell.
	# sum in depth:
	vhrho_nt = vhrho
	uhrho_et = uhrho
	# convert to transport:
	vhrho_nt = vhrho_nt*dxu/rho_0
	uhrho_et = uhrho_et*dyt/rho_0
	
	# overwrite coords, so we can add the freezing point (with uhrho_et and vhrho_nt) without problems:
	yu_ocean = cc.querying.getvar(exp,'yu_ocean',session,n=1)
	yu_ocean = yu_ocean.sel(yu_ocean=lat_range)
	#vhrho_nt.coords['yt_ocean'] = yu_ocean.values
	#vhrho_nt = vhrho_nt.rename(({'yt_ocean':'yu_ocean'}))
	#uhrho_et.coords['xt_ocean'] = xu_ocean.values
	#uhrho_et = uhrho_et.rename(({'xt_ocean':'xu_ocean'}))
	
	freezing_point_heat_trans_zonal = cp*rho_0*uhrho_et*temp_freezing
	freezing_point_heat_trans_meridional = cp*rho_0*vhrho_nt*temp_freezing
	temp_xflux = cp*rho_0*uhrho_et*tempx
	temp_yflux = cp*rho_0*vhrho_nt*tempy

In [32]:
	
	print("Removing freezing point heat transport")
	# compare both ways:
	temp_yflux_month = temp_yflux - freezing_point_heat_trans_meridional
	temp_xflux_month = temp_xflux - freezing_point_heat_trans_zonal
	
	# temp_yflux_with_mask_month = np.multiply(np.array(temp_yflux),np.array(mask_y_transport))
	# temp_xflux_with_mask_month = np.multiply(np.array(temp_xflux),np.array(mask_x_transport))
	
	
	#ndays_month=int(monthdays[int(int(2)-1)]) ###############################################################
	#print('number of days in the current month = ' +str(ndays_month))
	

Removing freezing point heat transport


In [13]:
	
	cutout_latind=510
	

In [14]:
	
	########### Import Adeles 1km contour on X,Y grid                
	outfile = '/g/data/v45/akm157/model_data/access-om2/Antarctic_slope_contour_'+str(isobath_depth)+'m.npz'
	data = np.load(outfile)
	mask_y_transport = data['mask_y_transport']; 
	mask_y_transport=mask_y_transport[:cutout_latind,:]       ##############################
	mask_x_transport = data['mask_x_transport']; 
	mask_x_transport=mask_x_transport[:cutout_latind,:]		################################
	mask_y_transport_numbered = data['mask_y_transport_numbered']
	mask_y_transport_numbered=mask_y_transport_numbered[:cutout_latind,:]		################################
	mask_x_transport_numbered = data['mask_x_transport_numbered']
	mask_x_transport_numbered=mask_x_transport_numbered[:cutout_latind,:]		################################
	
	
	#cutting the matrices
	ylength= np.shape(mask_x_transport)[0]
	
	
	yt_ocean0 = cc.querying.getvar(exp,'yt_ocean',session,n=1)
	yt_ocean0 = yt_ocean0.isel(yt_ocean=slice(0,ylength))
	yu_ocean0 = cc.querying.getvar(exp,'yu_ocean',session,n=1)
	yu_ocean0 = yu_ocean0.isel(yu_ocean=slice(0,ylength))
	xt_ocean0 = cc.querying.getvar(exp,'xt_ocean',session,n=1)
	xu_ocean0 = cc.querying.getvar(exp,'xu_ocean',session,n=1)
	

In [15]:
	
	mask_x_transport =xr.DataArray(data['mask_x_transport']).assign_coords({"dim_0": np.array(yt_ocean0),"dim_1": np.array(xu_ocean0)}).rename(dim_0="y_ocean",dim_1="x_ocean")
	mask_x_transport=mask_x_transport[:cutout_latind,:]       ##############################
	mask_y_transport =xr.DataArray(data['mask_y_transport']).assign_coords({"dim_0": np.array(yu_ocean0),"dim_1": np.array(xt_ocean0)}).rename(dim_0="y_ocean",dim_1="x_ocean")
	mask_y_transport=mask_y_transport[:cutout_latind,:]		################################		
	mask_x_transport_numbered =xr.DataArray(data['mask_x_transport_numbered']).assign_coords({"dim_0": np.array(yt_ocean0),"dim_1": np.array(xu_ocean0)}).rename(dim_0="y_ocean",dim_1="x_ocean")          ##################
	mask_x_transport_numbered=mask_x_transport_numbered[:cutout_latind,:]		################################		
	mask_y_transport_numbered =xr.DataArray(data['mask_y_transport_numbered']).assign_coords({"dim_0": np.array(yu_ocean0),"dim_1": np.array(xt_ocean0)}).rename(dim_0="y_ocean",dim_1="x_ocean")          ##################
	mask_y_transport_numbered=mask_y_transport_numbered[:cutout_latind,:]		################################
	
	num_points = int(np.maximum(np.max(mask_y_transport_numbered),np.max(mask_x_transport_numbered)))                
	
	
	
	
	
	############### Getting the lon,lat along contour in the X,Y contour
	lat_along_contour = np.zeros((num_points))
	lon_along_contour = np.zeros((num_points))
	# locations for zonal transport:
	x_indices_masked = mask_x_transport_numbered.stack().values
	x_indices = np.sort(x_indices_masked[x_indices_masked>0])
	for count in x_indices:
		count = int(count)
		jj = int(np.where(mask_x_transport_numbered==count)[0])
		ii = int(np.where(mask_x_transport_numbered==count)[1])   
		lon_along_contour[count-1] = xu_ocean0[ii].values
		lat_along_contour[count-1] = mask_x_transport_numbered.y_ocean[jj].values
		
	# locations for meridional transport:
	y_indices_masked = mask_y_transport_numbered.stack().values
	y_indices = np.sort(y_indices_masked[y_indices_masked>0])
	for count in y_indices:
		count = int(count)
		jj = np.where(mask_y_transport_numbered==count)[0]
		ii = np.where(mask_y_transport_numbered==count)[1]
		lon_along_contour[count-1] = mask_x_transport_numbered.x_ocean[ii].values           ##################          ##################          ##################          ##################          ##################          ##################
		lat_along_contour[count-1] = yu_ocean0[jj].values
	

In [16]:
	
	temp_yflux_with_mask_month=temp_yflux_month
	temp_yflux_with_mask_month['yu_ocean'] = np.array(mask_y_transport.y_ocean)
	temp_yflux_with_mask_month['xt_ocean'] = np.array(mask_y_transport.x_ocean)
	temp_yflux_with_mask_month = temp_yflux_with_mask_month.rename(yu_ocean='y_ocean',xt_ocean='x_ocean')*mask_y_transport
	
	temp_xflux_with_mask_month=temp_xflux_month
	temp_xflux_with_mask_month['yt_ocean'] = np.array(mask_y_transport.y_ocean)
	temp_xflux_with_mask_month['xu_ocean'] = np.array(mask_y_transport.x_ocean)
	temp_xflux_with_mask_month = temp_xflux_with_mask_month.rename(yt_ocean='y_ocean',xu_ocean='x_ocean')*mask_y_transport
	
	
	temp_yflux_with_mask_month['y_ocean']=np.array(mask_y_transport_numbered.y_ocean)
	temp_yflux_with_mask_month['x_ocean']=np.array(mask_y_transport_numbered.x_ocean)
	
	temp_xflux_with_mask_month['y_ocean']=np.array(mask_x_transport_numbered.y_ocean)
	temp_xflux_with_mask_month['x_ocean']=np.array(mask_x_transport_numbered.x_ocean)
	
	

Do not need to add this line as I loaded the daa before

In [17]:
	# # #there is a new problem with dimensions here. what is it? i dunno, gotta freaking check each dimension
	# temp_yflux_month = temp_yflux_month.load()
	# temp_xflux_month = temp_xflux_month.load()

In [18]:
	
	# Create the contour order data-array. Note that in this procedure the x-grid counts have x-grid
	#   dimensions and the y-grid counts have y-grid dimensions, but these are implicit, the dimension 
	#   *names* are kept general across the counts, the generic y_ocean, x_ocean, so that concatening works
	#   but we dont double up with numerous counts for one lat/lon point.
	
	# stack contour data into 1d:
	mask_x_numbered_1d = mask_x_transport_numbered.stack(contour_index = ['y_ocean', 'x_ocean'])
	mask_x_numbered_1d = mask_x_numbered_1d.where(mask_x_numbered_1d > 0, drop = True)
	
	mask_y_numbered_1d = mask_y_transport_numbered.stack(contour_index = ['y_ocean', 'x_ocean'])
	mask_y_numbered_1d = mask_y_numbered_1d.where(mask_y_numbered_1d > 0, drop = True)
	
	contour_ordering = xr.concat((mask_x_numbered_1d, mask_y_numbered_1d), dim = 'contour_index')
	contour_ordering = contour_ordering.sortby(contour_ordering)
	contour_index_array = np.arange(1, len(contour_ordering)+1)
	

for temp_xflux

In [33]:
	
	temp_yflux_month['yu_ocean']=np.array(mask_y_transport_numbered.y_ocean)
	temp_yflux_month['xt_ocean']=np.array(mask_y_transport_numbered.x_ocean)
	
	temp_xflux_month['yt_ocean']=np.array(mask_x_transport_numbered.y_ocean)
	temp_xflux_month['xu_ocean']=np.array(mask_x_transport_numbered.x_ocean)
	

In [34]:
	
	# stack transports into 1d and drop any points not on contour:
	x_along_1d = mask_x_transport.stack(contour_index = ['y_ocean', 'x_ocean'])
	x_along_1d = x_along_1d.where(mask_x_numbered_1d>0, drop = True)
	y_along_1d = mask_y_transport.stack(contour_index = ['y_ocean', 'x_ocean'])
	y_along_1d = y_along_1d.where(mask_y_numbered_1d>0, drop = True)
	
	# combine all points on contour, to obtain a matrix with transport directions:
	transp_dir = xr.concat((x_along_1d, y_along_1d), dim = 'contour_index')
	transp_dir = transp_dir.sortby(contour_ordering)
	transp_dir.coords['contour_index'] = contour_index_array
	transp_dir = transp_dir.load()
	

In [39]:
temp_xflux_month = temp_xflux_month.drop('xt_ocean')
temp_xflux_month = temp_xflux_month.drop('yu_ocean')

In [42]:
temp_yflux_month = temp_yflux_month.drop('yt_ocean')
temp_yflux_month = temp_yflux_month.drop('xu_ocean')

In [43]:
	
	## We could also loop in time if we didn't want the time average. Initialise a data array and fill in data by looping in time.
	# stack transports into 1d and drop any points not on contour:
	x_along_1d = temp_xflux_month.temp.rename(yt_ocean='y_ocean',xu_ocean='x_ocean').stack(contour_index = ['y_ocean', 'x_ocean'])
	x_along_1d = x_along_1d.where(mask_x_numbered_1d>0, drop = True)
	y_along_1d = temp_yflux_month.temp.rename(yu_ocean='y_ocean',xt_ocean='x_ocean').stack(contour_index = ['y_ocean', 'x_ocean'])
	y_along_1d = y_along_1d.where(mask_y_numbered_1d>0, drop = True)
	
	# combine all points on contour:
	heat_transp_across_contour = xr.concat((x_along_1d, y_along_1d), dim = 'contour_index')
	heat_transp_across_contour = heat_transp_across_contour.sortby(contour_ordering)
	heat_transp_across_contour.coords['contour_index'] = contour_index_array
	heat_transp_across_contour = (heat_transp_across_contour*transp_dir).load()
	

In [47]:
heat_transp_across_contour

<xarray.DataArray (month: 12, st_ocean: 75, contour_index: 6002)>
array([[[            nan, -5.42339734e+09, -5.07088600e+09, ...,
         -5.90523471e+09, -5.43151036e+09,             nan],
        [            nan, -6.15229650e+09, -5.76369980e+09, ...,
         -6.67624385e+09, -6.15416030e+09,             nan],
        [            nan, -6.78305223e+09, -6.35572244e+09, ...,
         -7.35656811e+09, -6.78107777e+09,             nan],
        ...,
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan]],

       [[            nan, -4.60798388e+09, -4.30033033e+09, ...,
         -5.35867102e+09, -4.91824669e+09,             nan],
        [            nan, -5.43671687e+09, -5.10303370e+09, ...,
         -6.24407870e+09, -5.76029373e+09,             nan],
        [            nan, -6.14277277e+09, -5.77730903e+09, ...,
         -7.02216103e+09, -6.48882087e+09,             nan],
...
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan]],

       [[            nan, -3.66150349e+09, -3.54552107e+09, ...,
         -3.92111338e+09, -3.69085228e+09,             nan],
        [            nan, -4.00505392e+09, -3.87499952e+09, ...,
         -4.29511930e+09, -4.03817892e+09,             nan],
        [            nan, -4.30006773e+09, -4.15348249e+09, ...,
         -4.62512492e+09, -4.33864170e+09,             nan],
        ...,
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan],
        [            nan,             nan,             nan, ...,
                     nan,             nan,             nan]]])
Coordinates:
  * st_ocean       (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * month          (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
    TLON           (contour_index) float64 80.05 80.15 80.25 ... 79.85 79.95 nan
    TLAT           (contour_index) float64 -66.01 -66.01 -66.01 ... -66.05 nan
    ULON           (contour_index) float64 -279.9 -279.8 -279.7 ... 80.0 nan
    ULAT           (contour_index) float64 -65.99 -65.99 -65.99 ... -66.03 nan
  * contour_index  (contour_index) int64 1 2 3 4 5 ... 5998 5999 6000 6001 6002

In [48]:
	
	# convert to longitude coordinate and average into 3 degree longitude bins:
	# in degrees:
	bin_width = 3
	bin_spacing = 0.25
	lon_west = -280
	lon_east = 80
	
	# new coordinate and midpoints of longitude bins:
	full_lon_coord = np.arange(lon_west,lon_east+bin_spacing,bin_spacing)
	lon_bin_midpoints = np.arange(lon_west+bin_width/2,lon_east-bin_width/2,bin_spacing)
	n_bin_edges = len(full_lon_coord)
	
	# sum into longitude bins:
	# need to be very careful of loops, we can't just mask over longitude values, but instead pick indices 
	# on the isobath contour and sum continously along contour between defined indices.
	# (i.e. lon_along_contour is not monotonic)
	# find points on contour to define edges of longitude bins:
	bin_edge_indices = np.zeros(n_bin_edges)
	for lon_bin in range(n_bin_edges-1):
		# find first isobath point that has the right longitude:
		first_point = np.where(lon_along_contour>=full_lon_coord[lon_bin])[0][0]
		# then find all other isobath points with the same longitude as that first point:
		same_lon_points = np.where(lon_along_contour==lon_along_contour[first_point])[0]
		# we want the most southerly of these points on the same longitude line:
		bin_edge_indices[lon_bin] = same_lon_points[np.argmin(lat_along_contour[same_lon_points])]
		
	# define east/west edges:
	bin_edge_indices = bin_edge_indices.astype(int)
	bin_edge_indices_west = bin_edge_indices[:-int(bin_width/bin_spacing)-1]
	bin_edge_indices_east = bin_edge_indices[int(bin_width/bin_spacing):-1]
	n_bins = len(bin_edge_indices_west)
	
	
	lat_bin_midpoints = np.zeros(n_bins)
	for lon_bin in range(n_bins):
		# find nearest isobath point:
		lon_index = np.where(lon_along_contour>=lon_bin_midpoints[lon_bin])[0][0]
		lat_bin_midpoints[lon_bin] = lat_along_contour[lon_index]
	

In [50]:
temp_xflux_month = temp_xflux_month.temp
temp_yflux_month = temp_yflux_month.temp

In [53]:
	
	# sum heat transport from isobath coord into new longitude coord:
	ndays_month=12
	cross_slope_heat_trans = np.zeros([ndays_month,75,n_bins])
	for lon_bin in range(n_bins):
		heat_trans_this_bin0 = heat_transp_across_contour[:,:,bin_edge_indices_west[lon_bin]:bin_edge_indices_east[lon_bin]]
		cross_slope_heat_trans[:,:,lon_bin] = np.sum(heat_trans_this_bin0,axis=2)
	
	#cross_slope_heat_trans=np.zeros([ndays_month,75,int(np.size(lon_bin_midpoints))])	
	#cross_slope_heat_trans[ndindex][nzindex]=cross_slope_heat_trans0
	# find average latitude of each bin, so we can plot back on the isobath:
	lat_bin_midpoints = np.zeros(n_bins)
	for lon_bin in range(n_bins):
		# find nearest isobath point:
		lon_index = np.where(lon_along_contour>=lon_bin_midpoints[lon_bin])[0][0]
		lat_bin_midpoints[lon_bin] = lat_along_contour[lon_index]
	
	
	print('Calculating Zonal heat convergence')
	# Need to make sure the zonal boundaries here match exactly with the zonal boundaries used for the 
	# longitude averaging above, by using same bin_edge_indices.
	# Just check if isobath point is on x or y grid. if x, sum up to this point from south.
	# if on y grid, sum zonal transport on nearest u grid point to west.
	
	zonal_heat_trans_west = np.zeros([ndays_month,75,n_bins])
	for lon_bin in range(n_bins):
		# west limit:
		# reset these to False:
		on_x_grid = False
		on_y_grid = False
		# mask_x_transport_numbered etc indexing starts from 1 not 0, so add 1:
		isobath_west_index = int(bin_edge_indices_west[lon_bin]+1)
		# check if the point is on the x or y transport grid:
		if len(np.where(mask_x_transport_numbered==isobath_west_index)[0])>0:
			on_x_grid = True
			jj = int(np.where(mask_x_transport_numbered==isobath_west_index)[0])
			ii = int(np.where(mask_x_transport_numbered==isobath_west_index)[1])
		elif len(np.where(mask_y_transport_numbered==isobath_west_index)[0])>0:
			on_y_grid = True
			jj = int(np.where(mask_y_transport_numbered==isobath_west_index)[0])
			ii = int(np.where(mask_y_transport_numbered==isobath_west_index)[1])               
		if on_x_grid == True:
			zonal_heat_trans_west[:,:,lon_bin] = np.sum(temp_xflux_month[:,:,:jj,ii],axis=2)
		#in this case we want transport half a grid point to the west:
		elif on_y_grid == True:
			# careful if ii=0, then we need heat trans from lon=80, because at limit of zonal grid
			if ii==0:
				zonal_heat_trans_west[:,:,lon_bin] = np.sum(temp_xflux_month[:,:,:jj+1,-1],axis=2)
			else:
				zonal_heat_trans_west[:,:,lon_bin] = np.sum(temp_xflux_month[:,:,:jj+1,ii-1],axis=2)
	
	zonal_heat_trans_east = np.zeros([ndays_month,75,n_bins])
	for lon_bin in range(n_bins):
		# east limit:
		# reset these to False:
		on_x_grid = False
		on_y_grid = False
		# mask_x_transport_numbered etc indexing starts from 1 not 0, so add 1:
		isobath_east_index = int(bin_edge_indices_east[lon_bin]+1)
		# check if the point is on the x or y transport grid:
		if len(np.where(mask_x_transport_numbered==isobath_east_index)[0])>0:
			on_x_grid = True
			jj = int(np.where(mask_x_transport_numbered==isobath_east_index)[0])
			ii = int(np.where(mask_x_transport_numbered==isobath_east_index)[1])
		elif len(np.where(mask_y_transport_numbered==isobath_east_index)[0])>0:
			on_y_grid = True
			jj = int(np.where(mask_y_transport_numbered==isobath_east_index)[0])
			ii = int(np.where(mask_y_transport_numbered==isobath_east_index)[1])
		if on_x_grid == True:
			zonal_heat_trans_east[:,:,lon_bin] = np.sum(temp_xflux_month[:,:,:jj,ii],axis=2)
			# in this case we want transport half a grid point to the west:
		elif on_y_grid == True:
			# wrap around to east side of grid:
			if ii==0:
				zonal_heat_trans_east[:,:,lon_bin] = np.sum(temp_xflux_month[:,:,:jj+1,-1],axis=2)
			else:
				zonal_heat_trans_east[:,:,lon_bin] = np.sum(temp_xflux_month[:,:,:jj+1,ii-1],axis=2)
	
	zonal_convergence = zonal_heat_trans_east - zonal_heat_trans_west
	
	
	
	

Calculating Zonal heat convergence


## Saving

In [59]:
	
	# PREVIOUS SAVEDIR TO LOCATE THE FILES save_dir  = '/g/data/x77/wf4500/ASC_project/ASC_speed/monthly/OM2_RYF/'
	save_dir  = '/home/156/wf4500/x77_wf4500/ASC_project/Post_process/IAF_CSHT_monthly_climatology.nc'
	
	
	data_ZC=xr.DataArray((zonal_convergence),name="zonal_convergence",dims=["month","st_ocean","lon_bin_midpoints"])
	data_CSHT=xr.DataArray((cross_slope_heat_trans),name="binned_cross_slope_heat_trans",dims=["month","st_ocean","lon_bin_midpoints"])
	data_HT=heat_transp_across_contour
	data_HT.name='unbinned_heat_transp_across_contour'
	data_lat=xr.DataArray((lat_along_contour),name="lat_along_contour",dims=["contour_index"])
	data_lon=xr.DataArray((lon_along_contour),name="lon_along_contour",dims=["contour_index"])
	data_latbin=xr.DataArray((lat_bin_midpoints),name="lat_bin_midpoints",dims=["lon_bin_midpoints"])
	
	
	
	#data_time=xr.DataArray(Ualong.time,name="time",dims=["time"])
	
	data_to_saveuv = xr.merge([data_ZC,data_CSHT,data_HT,data_lat,data_lon,data_latbin])
	data_to_saveuv['contour_index']=heat_transp_across_contour.contour_index
	data_to_saveuv['st_ocean']=np.array(heat_transp_across_contour.st_ocean)
	data_to_saveuv['lon_bin_midpoints']=np.array(lon_bin_midpoints)
	data_to_saveuv['month']=heat_transp_across_contour.month
	data_to_saveuv.to_netcdf(save_dir)
	
	print('Finished successful')
	

Finished successful


### 